## Data Coalition of fixtures and league tables

Within this workbook all the premier league fixtures from 1993 to 2018 and all the premier league tables during the period are combined into one usable csv.
As well as this some useful columns have been added which may aid in finding a link between the league results. These are the difference in league position, the form over the past 2,3,4,5 games of the team whose result is being calculated.
Other columns which could be added are the form of the opposition and the difference between the forms of the two sides.

Throughout the workbook there are various comments and also the data is regularly displayed so as to show th echanges that have been made. I have highlighted these where useful.

In [73]:
#here I just import the basic packages that will be used in theis worksheet
#Numpy and Pandas for there usual data analysis and datetime to help catorgarise the date string into a date easier.
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns

# Premier league Games

Start by importing the Premier league game data so we can see what we are working with, this is a basic set downloaded from Kaggle.

In [2]:
LeagueGames = pd.read_csv("EPL_Set.csv")
#LeagueGames is a DataFrame from pandas.
LeagueGames

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Season
0,E0,14/08/93,Arsenal,Coventry,0,3,A,NaN,NaN,NaN,1993-94
1,E0,14/08/93,Aston Villa,QPR,4,1,H,NaN,NaN,NaN,1993-94
2,E0,14/08/93,Chelsea,Blackburn,1,2,A,NaN,NaN,NaN,1993-94
3,E0,14/08/93,Liverpool,Sheffield Weds,2,0,H,NaN,NaN,NaN,1993-94
4,E0,14/08/93,Man City,Leeds,1,1,D,NaN,NaN,NaN,1993-94
...,...,...,...,...,...,...,...,...,...,...,...
9659,E0,13/05/18,Newcastle,Chelsea,3,0,H,1.0,0.0,H,2017-18
9660,E0,13/05/18,Southampton,Man City,0,1,A,0.0,0.0,D,2017-18
9661,E0,13/05/18,Swansea,Stoke,1,2,A,1.0,2.0,A,2017-18
9662,E0,13/05/18,Tottenham,Leicester,5,4,H,1.0,2.0,A,2017-18


Now we need to remove any non useful data, is is unlikely half time data will be relevant in biulding our model, so this will be removed. 

The division also shouldn't be useful as all games are expected to be in the Premier league, but this will be checked before deleting. It is checked below to ensure there is only one value which will be the premier league if true.

In [3]:
LeagueGames['Div'].unique()

array(['E0'], dtype=object)

Hence from the above it can be seen that only one value exists in the Div column and therefore it can be removed.

The goals will also be removed as it seems unlikely to be useful.

In [4]:
LeagueGames = LeagueGames.drop(['Div','FTHG','FTAG','HTHG','HTAG','HTR'],axis=1)
LeagueGames.head()

,Date,HomeTeam,AwayTeam,FTR,Season
0,14/08/93,Arsenal,Coventry,A,1993-94
1,14/08/93,Aston Villa,QPR,H,1993-94
2,14/08/93,Chelsea,Blackburn,A,1993-94
3,14/08/93,Liverpool,Sheffield Weds,H,1993-94
4,14/08/93,Man City,Leeds,D,1993-94


The "Season Column will be converted into something more useable such as seasonStart and seasonEnd.
This will allow an easier comparison to the final league table from the season before.


In [5]:
LeagueGames['seasonStart'] = LeagueGames['Season'].apply(lambda title: title.split('-')[0])
LeagueGames['seasonEnd'] = LeagueGames['Season'].apply(lambda title: title.split('-')[1])
LeagueGames['seasonEnd'] = LeagueGames['seasonEnd'].astype(int)
LeagueGames['seasonStart'] = LeagueGames['seasonStart'].astype(int)
LeagueGames['seasonEnd'] = np.where(LeagueGames['seasonEnd']<=20,2000+LeagueGames['seasonEnd'],
                                    1900+LeagueGames['seasonEnd'])

# A simple Lambda expression was used to split the season in 2 and then the first and 
# second part were taken for the start and end respectively
# These were then converted from strings to int types, to allow easier subtraction and sorting later.

LeagueGames.tail(1)

,Date,HomeTeam,AwayTeam,FTR,Season,seasonStart,seasonEnd
9663,13/05/18,West Ham,Everton,H,2017-18,2017,2018


Now because all results need to be considered, the table will be duplicated and every game duplicated, the new columns will have team, home/away, opponent, result, season. The reason for the duplication is that currently every team appears once for each fixture but it would be easier to double each fixture. Then each team appears twice for the same fixture, once as the team whose result is being modelled and once as the opposition.

This way it will be easier to analyse the data and the teams league position as the team and the opposition

In [6]:
LeagueGamesHome = LeagueGames
LeagueGamesHome['team'] = LeagueGamesHome['HomeTeam']
LeagueGamesHome['opposition'] = LeagueGamesHome['AwayTeam']
LeagueGamesHome = LeagueGamesHome.drop(['HomeTeam','AwayTeam'],axis=1)
LeagueGamesHome['result'] = np.where(LeagueGamesHome['FTR']=='H', 'Win',np.where(LeagueGamesHome['FTR']=='A','Loss','Draw'))
LeagueGamesHome['ho/Aw'] = 'Home'
LeagueGamesHome.tail()

,Date,FTR,Season,seasonStart,seasonEnd,team,opposition,result,ho/Aw
9659,13/05/18,H,2017-18,2017,2018,Newcastle,Chelsea,Win,Home
9660,13/05/18,A,2017-18,2017,2018,Southampton,Man City,Loss,Home
9661,13/05/18,A,2017-18,2017,2018,Swansea,Stoke,Loss,Home
9662,13/05/18,H,2017-18,2017,2018,Tottenham,Leicester,Win,Home
9663,13/05/18,H,2017-18,2017,2018,West Ham,Everton,Win,Home


In [7]:
LeagueGamesAway = LeagueGames
LeagueGamesAway['team'] = LeagueGamesAway['AwayTeam']
LeagueGamesAway['opposition'] = LeagueGamesAway['HomeTeam']
LeagueGamesAway = LeagueGamesAway.drop(['HomeTeam','AwayTeam'],axis=1)
LeagueGamesAway['result'] = np.where(LeagueGamesAway['FTR']=='H', 'Loss',np.where(LeagueGamesAway['FTR']=='A','Win','Draw'))
LeagueGamesAway['ho/Aw'] = 'Away'
LeagueGamesAway.head()

,Date,FTR,Season,seasonStart,seasonEnd,team,opposition,result,ho/Aw
0,14/08/93,A,1993-94,1993,1994,Coventry,Arsenal,Win,Away
1,14/08/93,H,1993-94,1993,1994,QPR,Aston Villa,Loss,Away
2,14/08/93,A,1993-94,1993,1994,Blackburn,Chelsea,Win,Away
3,14/08/93,H,1993-94,1993,1994,Sheffield Weds,Liverpool,Loss,Away
4,14/08/93,D,1993-94,1993,1994,Leeds,Man City,Draw,Away


In [8]:
LeagueGamesCombo = pd.concat([LeagueGamesHome,LeagueGamesAway])

#This just simply combines the two created dataframes which were duplcuates reversed into one listing.
#This listing has the columns identified below

LeagueGamesCombo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19328 entries, 0 to 9663
Data columns (total 9 columns):
Date           19328 non-null object
FTR            19328 non-null object
Season         19328 non-null object
seasonStart    19328 non-null int32
seasonEnd      19328 non-null int32
team           19328 non-null object
opposition     19328 non-null object
result         19328 non-null object
ho/Aw          19328 non-null object
dtypes: int32(2), object(7)
memory usage: 1.3+ MB


In [9]:
#Now attention is turned to the date column which as can be seen from looking at the type below is currently just an object.
#It's easier to work with a datetime object so it will be converted.

LeagueGamesCombo['Date'].tail()

9659    13/05/18
9660    13/05/18
9661    13/05/18
9662    13/05/18
9663    13/05/18
Name: Date, dtype: object

In [10]:
LeagueGamesCombo['date'] =  pd.to_datetime(LeagueGamesCombo['Date'],dayfirst=True)
LeagueGamesCombo = LeagueGamesCombo.drop(['Date','FTR'],axis=1)

# Then the old date column and the FTR column are removed as they are obsolete

LeagueGamesCombo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19328 entries, 0 to 9663
Data columns (total 8 columns):
Season         19328 non-null object
seasonStart    19328 non-null int32
seasonEnd      19328 non-null int32
team           19328 non-null object
opposition     19328 non-null object
result         19328 non-null object
ho/Aw          19328 non-null object
date           19328 non-null datetime64[ns]
dtypes: datetime64[ns](1), int32(2), object(5)
memory usage: 1.2+ MB


In [11]:
LeagueGamesCombo.columns=LeagueGamesCombo.columns.str.strip()
LeagueGamesCombo = LeagueGamesCombo.sort_values(['team','date'])
LeagueGamesCombo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19328 entries, 0 to 7383
Data columns (total 8 columns):
Season         19328 non-null object
seasonStart    19328 non-null int32
seasonEnd      19328 non-null int32
team           19328 non-null object
opposition     19328 non-null object
result         19328 non-null object
ho/Aw          19328 non-null object
date           19328 non-null datetime64[ns]
dtypes: datetime64[ns](1), int32(2), object(5)
memory usage: 1.2+ MB


Next it is important to establish the form of each team. For the purposes of this workbook form will consist of the number of point gained in the previous 2, 3, 4 and 5 games. If on an initial linear regression use it is found that one of these metrics is not useful it will be discarded.

Currently this is only for the team being focused on and also runs accross seasons which may or may not be helpful.

First we must establish the points gained by the 'Team' in each of their games which should be straightforward, a fixture number is also added so help keep track of each teams fixtures each season.

In [12]:
LeagueGamesCombo['teamPoints'] = np.where(LeagueGamesCombo['result']=='Win',3,np.where(LeagueGamesCombo['result']=='Draw',1,0))
LeagueGamesCombo = LeagueGamesCombo.reset_index()
LeagueGamesCombo = LeagueGamesCombo.drop('index',axis=1)

In [13]:
LeagueGamesCombo = LeagueGamesCombo.reset_index()
LeagueGamesCombo.head()

,index,Season,seasonStart,seasonEnd,team,opposition,result,ho/Aw,date,teamPoints
0,0,1993-94,1993,1994,Arsenal,Coventry,Loss,Home,1993-08-14,0
1,1,1993-94,1993,1994,Arsenal,Tottenham,Win,Away,1993-08-16,3
2,2,1993-94,1993,1994,Arsenal,Sheffield Weds,Win,Away,1993-08-21,3
3,3,1993-94,1993,1994,Arsenal,Leeds,Win,Home,1993-08-24,3
4,4,1993-94,1993,1994,Arsenal,Everton,Win,Home,1993-08-28,3


In [14]:
LeagueGamesCombo

,index,Season,seasonStart,seasonEnd,team,opposition,result,ho/Aw,date,teamPoints
0,0,1993-94,1993,1994,Arsenal,Coventry,Loss,Home,1993-08-14,0
1,1,1993-94,1993,1994,Arsenal,Tottenham,Win,Away,1993-08-16,3
2,2,1993-94,1993,1994,Arsenal,Sheffield Weds,Win,Away,1993-08-21,3
3,3,1993-94,1993,1994,Arsenal,Leeds,Win,Home,1993-08-24,3
4,4,1993-94,1993,1994,Arsenal,Everton,Win,Home,1993-08-28,3
...,...,...,...,...,...,...,...,...,...,...
19323,19323,2011-12,2011,2012,Wolves,Sunderland,Draw,Away,2012-04-14,1
19324,19324,2011-12,2011,2012,Wolves,Man City,Loss,Home,2012-04-22,0
19325,19325,2011-12,2011,2012,Wolves,Swansea,Draw,Away,2012-04-28,1
19326,19326,2011-12,2011,2012,Wolves,Everton,Draw,Home,2012-05-06,1


The below are functions designed to establish a teams form in the premier league exclusively. This takes the form of the number of points earned in the previous 'x' games. It currently runs from 2 to 5 as the method is not the most efficient meaning that bigger ones will take longer.

In [15]:
def find_last2(x):
    if x == 0:
        return 0
    elif x == 1:
        return LeagueGamesCombo['teamPoints'][x-1]
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-1]:
        return 0
    elif LeagueGamesCombo['team'][x-2] != LeagueGamesCombo['team'][x]:
        return LeagueGamesCombo['teamPoints'][x-1]
    else:
        return LeagueGamesCombo['teamPoints'][x-1]+LeagueGamesCombo['teamPoints'][x-2]
    
def find_last3(x):
    if x == 0:
        return 0
    elif x == 1:
        return LeagueGamesCombo['teamPoints'][x-1]
    elif x == 2:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2]
    
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-1]:
        return 0
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-2]:
        return LeagueGamesCombo['teamPoints'][x-1]
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-3]:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2]
    
    else:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2] + LeagueGamesCombo['teamPoints'][x-3]
    
def find_last4(x):
    if x == 0:
        return 0
    elif x == 1:
        return LeagueGamesCombo['teamPoints'][x-1]
    elif x == 2:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2]
    elif x == 3:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2] + + LeagueGamesCombo['teamPoints'][x-3]
    
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-1]:
        return 0
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-2]:
        return LeagueGamesCombo['teamPoints'][x-1]
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-3]:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2]
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-4]:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2] + LeagueGamesCombo['teamPoints'][x-3]
    
    else:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2] + LeagueGamesCombo['teamPoints'][x-3] + LeagueGamesCombo['teamPoints'][x-4]
    
def find_last5(x):
    if x == 0:
        return 0
    elif x == 1:
        return LeagueGamesCombo['teamPoints'][x-1]
    elif x == 2:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2]
    elif x == 3:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2] + + LeagueGamesCombo['teamPoints'][x-3]
    elif x == 4:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2] + + LeagueGamesCombo['teamPoints'][x-3] + LeagueGamesCombo['teamPoints'][x-4]
    
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-1]:
        return 0
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-2]:
        return LeagueGamesCombo['teamPoints'][x-1]
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-3]:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2]
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-4]:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2] + LeagueGamesCombo['teamPoints'][x-3]
    elif LeagueGamesCombo['team'][x] != LeagueGamesCombo['team'][x-5]:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2] + LeagueGamesCombo['teamPoints'][x-3] + LeagueGamesCombo['teamPoints'][x-4]
    
    else:
        return LeagueGamesCombo['teamPoints'][x-1] + LeagueGamesCombo['teamPoints'][x-2] + LeagueGamesCombo['teamPoints'][x-3] + LeagueGamesCombo['teamPoints'][x-4] + LeagueGamesCombo['teamPoints'][x-5]

The above method seemed really inefficent and on my laptop couldn't be run for instances longer than 8 in sensible time.
So I explored some new methods and designed a method which will work for n rows. This had significant design help from Tom Williamson, with the idea originally being his for a set n.

In [ ]:
def tw_find_sum_last_n(x,n):
    
    counter = 0
    
    if x == 0:
        return 0
    
    elif x < n:
        for i in range(0,x,1):
            if df['team'][x] != df['team'][i]:
                break
            counter +=df['teamPoints'][i]
    else:
        for i in range(x-1,x-n-1,-1):
            if df['team'][x] != df['team'][i]:
                break
            counter +=df['teamPoints'][i]
    
    return counter

In [16]:
LeagueGamesCombo['team_form_2games'] = LeagueGamesCombo['index'].apply(lambda x : find_last2(x))
LeagueGamesCombo['team_form_3games'] = LeagueGamesCombo['index'].apply(lambda x : find_last3(x))
LeagueGamesCombo['team_form_4games'] = LeagueGamesCombo['index'].apply(lambda x : find_last4(x))
LeagueGamesCombo['team_form_5games'] = LeagueGamesCombo['index'].apply(lambda x : find_last5(x))

Now it is also worth finding out the form of all the Opposition teams leading into each game as well to see if that has an impact on the result, it could be that a useful metric is instead to look at the difference in form between the two teams to see if this is a useful predictor.

All that needs to be done to find the opposition form is to find to corresponding reverse fixture where the "team" form will have ben found already

In [17]:
def find_form(x,y):
    finder = 'string'
    if y==2: finder = 'team_form_2games'
    elif y==3: finder = 'team_form_3games'
    elif y==4: finder = 'team_form_4games'
    elif y==5: finder = 'team_form_5games'
    else: raise Exception('y must be an integer between 2 and 5')
    
    team = LeagueGamesCombo['opposition'][x]
    date = LeagueGamesCombo['date'][x]
    result = LeagueGamesCombo[(LeagueGamesCombo['team'] == team) & (LeagueGamesCombo['date'] == date)][finder].iat[0]
    return result

In [18]:
LeagueGamesCombo['opp_form_2games'] = LeagueGamesCombo['index'].apply(lambda x,y=2 : find_form(x,y))
LeagueGamesCombo['opp_form_3games'] = LeagueGamesCombo['index'].apply(lambda x,y=3 : find_form(x,y))
LeagueGamesCombo['opp_form_4games'] = LeagueGamesCombo['index'].apply(lambda x,y=4 : find_form(x,y))
LeagueGamesCombo['opp_form_5games'] = LeagueGamesCombo['index'].apply(lambda x,y=5 : find_form(x,y))

Now 4 new columns will be added which represent the difference in form between the teams and a 5th which will be home or away for 'team' in a binary form

In [19]:
LeagueGamesCombo['diff_2_game_form'] = LeagueGamesCombo['team_form_2games'] - LeagueGamesCombo['opp_form_2games']
LeagueGamesCombo['diff_3_game_form'] = LeagueGamesCombo['team_form_3games'] - LeagueGamesCombo['opp_form_3games']
LeagueGamesCombo['diff_4_game_form'] = LeagueGamesCombo['team_form_4games'] - LeagueGamesCombo['opp_form_4games']
LeagueGamesCombo['diff_5_game_form'] = LeagueGamesCombo['team_form_5games'] - LeagueGamesCombo['opp_form_5games']
LeagueGamesCombo['ho/Aw_binary'] = np.where(LeagueGamesCombo['ho/Aw']=='Home',1,0)

# Premier League Tables

Now that we have our League games in a useable format it's time to also add in the league position that each team finished in the previous season. In addition to this each clubs form will need to be established.

In [20]:
LeagueTables = pd.read_csv("tables_1968_2019.csv")
LeagueTables.head()

,season,name,pos,team,p,w,d,l,f,a,gd,points
0,2018/2019,Premier League 2018-2019,1,Manchester City,38,32,2,4,95,23,72,98
1,2018/2019,Premier League 2018-2019,2,Liverpool,38,30,7,1,89,22,67,97
2,2018/2019,Premier League 2018-2019,3,Chelsea,38,21,9,8,63,39,24,72
3,2018/2019,Premier League 2018-2019,4,Tottenham Hotspur,38,23,2,13,67,39,28,71
4,2018/2019,Premier League 2018-2019,5,Arsenal,38,21,7,10,73,51,22,70


Now need to remove the non useful features, the season and teams and position will be used, but not the other features on this first run through, these columns should be removed for ease of use.

In [21]:
LeagueTables = LeagueTables.drop(['name','p','w','d','l','f','a','gd','points'],axis=1)

The rows which relate to seasons for which no data is present for game results should also be dropped, this means all years before the season 92/93.

So create a new column with the year the season started and ended

In [22]:
LeagueTables['seasonStart'] = LeagueTables['season'].apply(lambda title: title.split('/')[0])
LeagueTables['seasonEnd'] = LeagueTables['season'].apply(lambda title: title.split('/')[1])
LeagueTables.head()

,season,pos,team,seasonStart,seasonEnd
0,2018/2019,1,Manchester City,2018,2019
1,2018/2019,2,Liverpool,2018,2019
2,2018/2019,3,Chelsea,2018,2019
3,2018/2019,4,Tottenham Hotspur,2018,2019
4,2018/2019,5,Arsenal,2018,2019


Finally for ease of use with the data any data from before the 92/93 season can be discarded

In [23]:
LeagueTables['seasonEnd'] = LeagueTables['seasonEnd'].astype(int)
LeagueTables = LeagueTables[LeagueTables['seasonEnd'] >= 1993]
LeagueTables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 0 to 545
Data columns (total 5 columns):
season         546 non-null object
pos            546 non-null int64
team           546 non-null object
seasonStart    546 non-null object
seasonEnd      546 non-null int32
dtypes: int32(1), int64(1), object(3)
memory usage: 23.5+ KB


In [24]:
LeagueTables

,season,pos,team,seasonStart,seasonEnd
0,2018/2019,1,Manchester City,2018,2019
1,2018/2019,2,Liverpool,2018,2019
2,2018/2019,3,Chelsea,2018,2019
3,2018/2019,4,Tottenham Hotspur,2018,2019
4,2018/2019,5,Arsenal,2018,2019
...,...,...,...,...,...
541,1992/1993,18,Southampton,1992,1993
542,1992/1993,19,Oldham Athletic,1992,1993
543,1992/1993,20,Crystal Palace,1992,1993
544,1992/1993,21,Middlesbrough,1992,1993


# Combine Results and league Position

It is now time to combine the game results with each teams league position they finished the previous year in.

First it is important to consider is all teams have the same names in each dataset as they have come from different sources. The number of teams should stay the same. 

Below it is checked whether each source has the same number of teams and what their names are.

In [25]:
LTT = pd.Series(LeagueTables['team'].sort_values().unique())
LGT = pd.Series(LeagueGamesCombo['team'].sort_values().unique())
Check1 = pd.concat((LTT,LGT),axis=1)
Check1

,0,1
0,Arsenal,Arsenal
1,Aston Villa,Aston Villa
2,Barnsley,Barnsley
3,Birmingham City,Birmingham
4,Blackburn Rovers,Blackburn
5,Blackpool,Blackpool
6,Bolton Wanderers,Bolton
7,Bournemouth,Bournemouth
8,Bradford City,Bradford
9,Brighton,Brighton


From the above table we can see that the offending teams that are causing the differences are 'Leeds United','West Ham united' which appears on the league table side twice. To filter it out Leed united will be renamed Leeds in the LeagueTables dataframe.

On the League games side we can see that Middlesboro also appears twice so this should also be filtered out in favour of Middlesborough.

In [26]:
LeagueTables['team'] = np.where(LeagueTables['team'] == 'Leeds United', 'Leeds', LeagueTables['team'])
LeagueTables['team'] = np.where(LeagueTables['team'] == 'West Ham United', 'West Ham', LeagueTables['team'])
LeagueGamesCombo['team'] = np.where(LeagueGamesCombo['team'] == 'Middlesboro', 'Middlesbrough', LeagueGamesCombo['team'])
LeagueGamesCombo['opposition'] = np.where(LeagueGamesCombo['opposition'] == 'Middlesboro', 'Middlesbrough', LeagueGamesCombo['opposition'])
LeagueGamesCombo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19328 entries, 0 to 19327
Data columns (total 23 columns):
index               19328 non-null int64
Season              19328 non-null object
seasonStart         19328 non-null int32
seasonEnd           19328 non-null int32
team                19328 non-null object
opposition          19328 non-null object
result              19328 non-null object
ho/Aw               19328 non-null object
date                19328 non-null datetime64[ns]
teamPoints          19328 non-null int32
team_form_2games    19328 non-null int64
team_form_3games    19328 non-null int64
team_form_4games    19328 non-null int64
team_form_5games    19328 non-null int64
opp_form_2games     19328 non-null int64
opp_form_3games     19328 non-null int64
opp_form_4games     19328 non-null int64
opp_form_5games     19328 non-null int64
diff_2_game_form    19328 non-null int64
diff_3_game_form    19328 non-null int64
diff_4_game_form    19328 non-null int64
diff_5_game_form

Now check that the team names are almost consistent by reusing the same formula as before

In [27]:
LTT = pd.Series(LeagueTables['team'].sort_values().unique())
LGTt = pd.Series(LeagueGamesCombo['team'].sort_values().unique())
LGTo = pd.Series(LeagueGamesCombo['opposition'].sort_values().unique())
Check2 = pd.concat((LTT,LGTt,LGTo),axis=1,)
Check2

,0,1,2
0,Arsenal,Arsenal,Arsenal
1,Aston Villa,Aston Villa,Aston Villa
2,Barnsley,Barnsley,Barnsley
3,Birmingham City,Birmingham,Birmingham
4,Blackburn Rovers,Blackburn,Blackburn
5,Blackpool,Blackpool,Blackpool
6,Bolton Wanderers,Bolton,Bolton
7,Bournemouth,Bournemouth,Bournemouth
8,Bradford City,Bradford,Bradford
9,Brighton,Brighton,Brighton


Now change the League Table team names so that they match the LeagueGamesCombo Array, this will make it easier later when looking up league position.

In [28]:
check_dict = Check2.set_index(0)[1].to_dict()
LeagueTables = LeagueTables.replace(check_dict)

No do a final check to ensure that all team names are aligned

In [29]:
LTT = pd.Series(LeagueTables['team'].sort_values().unique())
LGTt = pd.Series(LeagueGamesCombo['team'].sort_values().unique())
LGTo = pd.Series(LeagueGamesCombo['team'].sort_values().unique())
Check3 = pd.concat((LTT,LGTt,LGTo),axis=1)
Check3['check'] = (Check3[0] == Check3[1]) & (Check3[0] == Check3[2])
Check3

,0,1,2,check
0,Arsenal,Arsenal,Arsenal,True
1,Aston Villa,Aston Villa,Aston Villa,True
2,Barnsley,Barnsley,Barnsley,True
3,Birmingham,Birmingham,Birmingham,True
4,Blackburn,Blackburn,Blackburn,True
5,Blackpool,Blackpool,Blackpool,True
6,Bolton,Bolton,Bolton,True
7,Bournemouth,Bournemouth,Bournemouth,True
8,Bradford,Bradford,Bradford,True
9,Brighton,Brighton,Brighton,True


Now it's important to actually use the league table in the previous season to guide the current seasons results.
So for the results in the 93/94 season we need the table from the 92/93.
So For the tables we will use the 'SeasonEnd' column and for the results we should use the Season 'SeasonStart' column. 

To do this a pivot table will be made.

In [30]:
LeagueTables.head(3)

,season,pos,team,seasonStart,seasonEnd
0,2018/2019,1,Man City,2018,2019
1,2018/2019,2,Liverpool,2018,2019
2,2018/2019,3,Chelsea,2018,2019


In [31]:
UnstackedLeagueTables = LeagueTables.pivot(index='team',columns='seasonEnd',values='pos').unstack().to_frame()
UnstackedLeagueTables.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1323 entries, (1993, Arsenal) to (2019, Wolves)
Data columns (total 1 columns):
0    546 non-null float64
dtypes: float64(1)
memory usage: 13.6+ KB


In [32]:
LeagueGamesCombo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19328 entries, 0 to 19327
Data columns (total 23 columns):
index               19328 non-null int64
Season              19328 non-null object
seasonStart         19328 non-null int32
seasonEnd           19328 non-null int32
team                19328 non-null object
opposition          19328 non-null object
result              19328 non-null object
ho/Aw               19328 non-null object
date                19328 non-null datetime64[ns]
teamPoints          19328 non-null int32
team_form_2games    19328 non-null int64
team_form_3games    19328 non-null int64
team_form_4games    19328 non-null int64
team_form_5games    19328 non-null int64
opp_form_2games     19328 non-null int64
opp_form_3games     19328 non-null int64
opp_form_4games     19328 non-null int64
opp_form_5games     19328 non-null int64
diff_2_game_form    19328 non-null int64
diff_3_game_form    19328 non-null int64
diff_4_game_form    19328 non-null int64
diff_5_game_form

In [33]:
LeagueGamesCombo1 = LeagueGamesCombo.merge(UnstackedLeagueTables, left_on=["seasonStart", "team"],
                                          right_on=['seasonEnd','team'],how='left')


In [34]:
LeagueGamesCombo1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19328 entries, 0 to 19327
Data columns (total 24 columns):
index               19328 non-null int64
Season              19328 non-null object
seasonStart         19328 non-null int32
seasonEnd           19328 non-null int32
team                19328 non-null object
opposition          19328 non-null object
result              19328 non-null object
ho/Aw               19328 non-null object
date                19328 non-null datetime64[ns]
teamPoints          19328 non-null int32
team_form_2games    19328 non-null int64
team_form_3games    19328 non-null int64
team_form_4games    19328 non-null int64
team_form_5games    19328 non-null int64
opp_form_2games     19328 non-null int64
opp_form_3games     19328 non-null int64
opp_form_4games     19328 non-null int64
opp_form_5games     19328 non-null int64
diff_2_game_form    19328 non-null int64
diff_3_game_form    19328 non-null int64
diff_4_game_form    19328 non-null int64
diff_5_game_form

In [35]:
LeagueGamesCombo2 = LeagueGamesCombo1.merge(UnstackedLeagueTables, left_on=["seasonStart", "opposition"],
                                          right_on=['seasonEnd','team'],how='left')

In [36]:
LeagueGamesCombo2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19328 entries, 0 to 19327
Data columns (total 25 columns):
index               19328 non-null int64
Season              19328 non-null object
seasonStart         19328 non-null int32
seasonEnd           19328 non-null int32
team                19328 non-null object
opposition          19328 non-null object
result              19328 non-null object
ho/Aw               19328 non-null object
date                19328 non-null datetime64[ns]
teamPoints          19328 non-null int32
team_form_2games    19328 non-null int64
team_form_3games    19328 non-null int64
team_form_4games    19328 non-null int64
team_form_5games    19328 non-null int64
opp_form_2games     19328 non-null int64
opp_form_3games     19328 non-null int64
opp_form_4games     19328 non-null int64
opp_form_5games     19328 non-null int64
diff_2_game_form    19328 non-null int64
diff_3_game_form    19328 non-null int64
diff_4_game_form    19328 non-null int64
diff_5_game_form

In [37]:
LeagueGamesCombo2 = LeagueGamesCombo2.rename(columns={"0_x": "py_T_Leag_Pos", "0_y": "py_O_Leag_Pos"})
LeagueGamesCombo2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19328 entries, 0 to 19327
Data columns (total 25 columns):
index               19328 non-null int64
Season              19328 non-null object
seasonStart         19328 non-null int32
seasonEnd           19328 non-null int32
team                19328 non-null object
opposition          19328 non-null object
result              19328 non-null object
ho/Aw               19328 non-null object
date                19328 non-null datetime64[ns]
teamPoints          19328 non-null int32
team_form_2games    19328 non-null int64
team_form_3games    19328 non-null int64
team_form_4games    19328 non-null int64
team_form_5games    19328 non-null int64
opp_form_2games     19328 non-null int64
opp_form_3games     19328 non-null int64
opp_form_4games     19328 non-null int64
opp_form_5games     19328 non-null int64
diff_2_game_form    19328 non-null int64
diff_3_game_form    19328 non-null int64
diff_4_game_form    19328 non-null int64
diff_5_game_form

In [38]:
LeagueGamesCombo2 = LeagueGamesCombo2.fillna(21)
LeagueGamesCombo2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19328 entries, 0 to 19327
Data columns (total 25 columns):
index               19328 non-null int64
Season              19328 non-null object
seasonStart         19328 non-null int32
seasonEnd           19328 non-null int32
team                19328 non-null object
opposition          19328 non-null object
result              19328 non-null object
ho/Aw               19328 non-null object
date                19328 non-null datetime64[ns]
teamPoints          19328 non-null int32
team_form_2games    19328 non-null int64
team_form_3games    19328 non-null int64
team_form_4games    19328 non-null int64
team_form_5games    19328 non-null int64
opp_form_2games     19328 non-null int64
opp_form_3games     19328 non-null int64
opp_form_4games     19328 non-null int64
opp_form_5games     19328 non-null int64
diff_2_game_form    19328 non-null int64
diff_3_game_form    19328 non-null int64
diff_4_game_form    19328 non-null int64
diff_5_game_form

A more useful measure than league position may be the difference in league position of the two sides so this metric will also be included.

In [39]:
LeagueGamesCombo2['leagueDifference'] = LeagueGamesCombo2['py_T_Leag_Pos'] - LeagueGamesCombo2['py_O_Leag_Pos']

In [48]:
LeagueGamesCombo2.head()

,index,Season,seasonStart,seasonEnd,team,opposition,result,ho/Aw,date,teamPoints,...,opp_form_5games,diff_2_game_form,diff_3_game_form,diff_4_game_form,diff_5_game_form,ho/Aw_binary,py_T_Leag_Pos,py_O_Leag_Pos,leagueDifference,monthNumber
0,0,1993-94,1993,1994,Arsenal,Coventry,Loss,Home,1993-08-14,0,...,0,0,0,0,0,1,10.0,15.0,-5.0,6
1,1,1993-94,1993,1994,Arsenal,Tottenham,Win,Away,1993-08-16,3,...,3,-3,-3,-3,-3,0,10.0,8.0,2.0,6
2,2,1993-94,1993,1994,Arsenal,Sheffield Weds,Win,Away,1993-08-21,3,...,1,2,2,2,2,0,10.0,7.0,3.0,6
3,3,1993-94,1993,1994,Arsenal,Leeds,Win,Home,1993-08-24,3,...,4,3,2,2,2,1,10.0,17.0,-7.0,6
4,4,1993-94,1993,1994,Arsenal,Everton,Win,Home,1993-08-28,3,...,9,3,3,0,0,1,10.0,13.0,-3.0,6


Another useful metric would be to see the month the game takes place in, the reasoning is that as the season goes on the league table predictor will become stronger and stronger

In [69]:
def monthConverter(month):
    if month > 7: 
        newMonth = month - 7
    elif month < 8: 
        newMonth = month + 5
    return newMonth

In [79]:
LeagueGamesCombo2['monthNumber'] = LeagueGamesCombo2['date'].apply(lambda x: x.month)
LeagueGamesCombo2['monthNumber'] = LeagueGamesCombo2['monthNumber'].apply(lambda x: monthConverter(x))

In [80]:
LeagueGamesCombo2.to_csv(r"C:\Users\Elliott\Documents\Python\PremierLeaguePredictor\Git\combined_data.csv")

#finally the dataframe is exported for use within another workbook where the data analysis and logistic/linear regaression can take place